In [1]:
import oda_api.token 
import logging
import numpy as np
from oda_api.api import DispatcherAPI
from oda_api.plot_tools import OdaImage, OdaLightCurve, OdaSpectrum
import matplotlib.pyplot as plt
import astroquery.heasarc
from astropy.wcs import WCS
from astropy.io import fits
from astroquery.simbad import Simbad
from astropy import coordinates as coord
from astropy.coordinates import SkyCoord
from astropy.time import Time
from matplotlib.patches import Circle
from astroquery.jplhorizons import Horizons
import pandas as pd
import astropy.units as u
import json
from collections import defaultdict
import os
from datetime import datetime

In [2]:
logging.getLogger().setLevel(logging.WARNING)
logging.getLogger('oda_api').addHandler(logging.StreamHandler())

Load the ScWs.

In [3]:
scw = []
start = []
end = []

with open("../data/Crab-ScWs.txt", "r") as f:
    next(f)
    for line in f:
        parts = line.strip().split(",")
        scw.append(parts[0])
        start.append(float(parts[1]))  
        end.append(float(parts[2]))  

unique_sorted_data = {}
for scw, start, end in sorted(zip(scw, start, end), key=lambda x: x[0]):
    if scw not in unique_sorted_data:  
        unique_sorted_data[scw] = (scw, start, end)

scw, start, end = map(list, zip(*unique_sorted_data.values()))
durations = [e-s for e,s in zip(end, start)]

In [4]:
crab = SkyCoord.from_name("Crab")
crab_ra = crab.ra.deg
crab_dec = crab.dec.deg

crab_coords = coord.SkyCoord(ra=crab_ra, dec=crab_dec, unit=(u.deg, u.deg))

## QUERYING 

As we can see, Crab observations only happen twice per year in 3-month groups (Feb->Apr and Aug->Oct).
The idea now will be to get some longterm lightcurves over the years and see how the countrate evolves. Depending on the variability of the countrate, we will use it for flux conversion.

## IMAGES

We use asynchronous querying. We aim to get one result per month, every year. Since the ScWs only happen in two 3-month intervals per year, this will mean 6 results every year for 22 years so 120 results total. 

To choose the ScW every month, we will take the shortest one for time reasons.

In [5]:
isot_start_times = Time(start, format='mjd').isot
isot_end_times = Time(end, format='mjd').isot
duration_seconds = [duration * 86400 for duration in durations]
year_months = [st[:7] for st in isot_start_times]

How many ScWs when filtering pointings and choosing only one per month?

In [6]:
scw_per_month = {}
scw_count_by_year_month = {}
filtered_scw_per_month = {}

for scw_id, ym in zip(scw, year_months):
    if ym not in scw_per_month:
        scw_per_month[ym] = scw_id  

print(f"Number of SCWs if picking one per month: {len(scw_per_month)}")

for year_month in sorted(set(year_months)):
    filtered_scws = [
        (scw[i], isot_start_times[i], isot_end_times[i])
        for i, year_month_in_list in enumerate(year_months)
        if year_month_in_list == year_month
    ]

    if not filtered_scws:
        scw_count_by_year_month[year_month] = 0
        continue

    rrrr_max_pppp = defaultdict(int)
    for scwgr in filtered_scws:
        rrrr = scwgr[0][:4]  
        pppp = int(scwgr[0][4:8])
        rrrr_max_pppp[rrrr] = max(rrrr_max_pppp[rrrr], pppp)

    filtered_scws = [
        scwind for scwind in filtered_scws
        if 5 < int(scwind[0][4:8]) < rrrr_max_pppp[scwind[0][:4]] - 10
    ]

    scw_count_by_year_month[year_month] = len(filtered_scws)

    # Check if the "one-per-month" SCW is in the filtered list
    if year_month in scw_per_month and scw_per_month[year_month] in [scwind[0] for scwind in filtered_scws]:
        filtered_scw_per_month[year_month] = scw_per_month[year_month]

print(f"Total SCWs after filtering: {sum(scw_count_by_year_month.values())}")
print(f"Total SCWs when picking one per month AND applying filtering: {len(filtered_scw_per_month)}")

Number of SCWs if picking one per month: 104
Total SCWs after filtering: 4931
Total SCWs when picking one per month AND applying filtering: 43


15 - 30 keV

In [ ]:
'''disp_by_date = {} 
data_by_date = {}
successful_scws = []

disp = DispatcherAPI(url="https://www.astro.unige.ch/mmoda/dispatch-data", instrument="mock")

while True:
    image_results = []

    for year_month in sorted(set(year_months)):
        filtered_scws = [
            (scw[i], isot_start_times[i], isot_end_times[i], duration_seconds[i])
            for i, year_month_in_list in enumerate(year_months)
            if year_month_in_list == year_month
        ]

        if not filtered_scws:
            print(f"No SCWs found for {year_month}")
            continue

        # add filtering for not choosing the first 5 and last 10 ScWs of each revolution 
        # (ScWs look like RRRRPPPPSSS.001, so choose 5 < PPPP < max(PPPP) - 10)
        rrrr_max_pppp = defaultdict(int)
        for scwgr in filtered_scws:
            rrrr = scwgr[0][:4]  
            pppp = int(scwgr[0][4:8])
            rrrr_max_pppp[rrrr] = max(rrrr_max_pppp[rrrr], pppp)

        filtered_scws = [
            scwind for scwind in filtered_scws
            if 5 < int(scwind[0][4:8]) < rrrr_max_pppp[scwind[0][:4]] - 10
        ]

        # Sort SCWs by duration (smallest first)
        filtered_scws.sort(key=lambda x: x[3]) 

        if not filtered_scws:
            print(f"No SCWs found for {year_month} after filtering for pointings")
            continue
        else:
            print(f"{len(filtered_scws)} SCWs found for {year_month} after filtering for pointings")

        for scw_id, start_time, end_time, duration in filtered_scws:
            print(f"Trying SCW {scw_id} with duration {duration} for {year_month}")

            par_dict = {
                "RA": crab_ra,
                "DEC": crab_dec,
                "E1_keV": "15",
                "E2_keV": "30",
                "T_format": "isot",
                'T1': start_time,
                'T2': end_time,
                "detection_threshold": "5",
                "instrument": "isgri",
                "osa_version": "OSA11.2",
                "product": "isgri_image",
                "product_type": "Real",
                "scw_list": [scw_id],
                #'token': disp.disable_email_token(oda_api.token.discover_token()),
            }

            if scw_id not in disp_by_date:
                disp_by_date[scw_id] = DispatcherAPI(url="https://www.astro.unige.ch/mmoda/dispatch-data", instrument="mock", wait=False)

            _disp = disp_by_date[scw_id]

            data = data_by_date.get(scw_id, None)

            if data is None and not _disp.is_failed:
                try:
                    if not _disp.is_submitted:
                        data = _disp.get_product(**par_dict, silent=True)
                    else:
                        _disp.poll()

                    if not _disp.is_complete:
                        # continue  # Retry with the next SCW
                        # raise ValueError("Query incomplete")
                        break

                    data = _disp.get_product(**par_dict, silent=True)
                    data_by_date[scw_id] = data
                    image_results.append(data)
                    successful_scws.append(scw_id)
                    print(f"Query successful for SCW {scw_id}")
                    break  # Stop trying other SCWs for this month

                except Exception as e:
                    print(f"Query failed for SCW {scw_id}: {e}")
                    continue  # Try the next SCW

        else:
            print(f"All SCWs failed for {year_month}, skipping.")
            # print(f"Skipping {year_month} due to errors.")

    n_complete = len([year for year, _disp in disp_by_date.items() if _disp.is_complete])
    print(f"complete {n_complete} / {len(disp_by_date)}")

    if n_complete == len(disp_by_date):
        print("done!")
        break
    print("not done")'''

30 - 60 keV

In [ ]:
'''disp_by_date = {} 
data_by_date = {}
successful_scws = []

disp = DispatcherAPI(url="https://www.astro.unige.ch/mmoda/dispatch-data", instrument="mock")

# Keep track of the year-months that have already been processed
processed_year_months = set()

while True:
    image_results = []

    for year_month in sorted(set(year_months)):
        #if year_month in processed_year_months:
            #print(f"Already processed {year_month}, skipping.")
            #continue  # Skip if this year-month has already been processed

        filtered_scws = [
            (scw[i], isot_start_times[i], isot_end_times[i], duration_seconds[i])
            for i, year_month_in_list in enumerate(year_months)
            if year_month_in_list == year_month
        ]

        #if not filtered_scws:
            #print(f"No SCWs found for {year_month}")
            #continue

        # Add filtering for not choosing the first 5 and last 10 SCWs of each revolution
        rrrr_max_pppp = defaultdict(int)
        for scwgr in filtered_scws:
            rrrr = scwgr[0][:4]  
            pppp = int(scwgr[0][4:8])
            rrrr_max_pppp[rrrr] = max(rrrr_max_pppp[rrrr], pppp)

        filtered_scws = [
            scwind for scwind in filtered_scws
            if 5 < int(scwind[0][4:8]) < rrrr_max_pppp[scwind[0][:4]] - 10
        ]

        # Sort SCWs by duration (smallest first)
        filtered_scws.sort(key=lambda x: x[3]) 

        #if not filtered_scws:
            #print(f"No SCWs found for {year_month} after filtering for pointings")
            #continue
        #else:
            #print(f"{len(filtered_scws)} SCWs found for {year_month} after filtering for pointings")

        # Process SCWs for the current year-month
        for scw_id, start_time, end_time, duration in filtered_scws:
            print(f"Trying SCW {scw_id} with duration {duration} for {year_month}")

            par_dict = {
                "RA": crab_ra,
                "DEC": crab_dec,
                "E1_keV": "30",
                "E2_keV": "60",
                "T_format": "isot",
                'T1': start_time,
                'T2': end_time,
                "detection_threshold": "5",
                "instrument": "isgri",
                "osa_version": "OSA11.2",
                "product": "isgri_image",
                "product_type": "Real",
                "scw_list": [scw_id],
            }

            if scw_id not in disp_by_date:
                disp_by_date[scw_id] = DispatcherAPI(url="https://www.astro.unige.ch/mmoda/dispatch-data", instrument="mock", wait=False)

            _disp = disp_by_date[scw_id]

            data = data_by_date.get(scw_id, None)

            if data is None and not _disp.is_failed:
                try:
                    if not _disp.is_submitted:
                        data = _disp.get_product(**par_dict, silent=True)
                    else:
                        _disp.poll()

                    if not _disp.is_complete:
                        break  # Continue to the next SCW if not complete

                    data = _disp.get_product(**par_dict, silent=True)
                    data_by_date[scw_id] = data
                    image_results.append(data)
                    successful_scws.append(scw_id)
                    #print(f"Query successful for SCW {scw_id}")
                    break  # Stop trying other SCWs for this month if one is successful

                except Exception as e:
                    #print(f"Query failed for SCW {scw_id}: {e}")
                    continue  # Try the next SCW

        # Mark the year-month as processed once all SCWs are attempted
        #processed_year_months.add(year_month)

        #print(f"Finished processing {year_month}. Moving to the next month.")

    # Now, poll all SCWs that have been submitted
    print("Polling for completion of all SCWs.")
    n_complete = 0

    # Poll all SCWs that have been submitted (skip already completed ones)
    for scw_id, _disp in disp_by_date.items():
        if _disp.is_complete and scw_id not in data_by_date:
            try:
                data = _disp.get_product(silent=True)
                data_by_date[scw_id] = data
                image_results.append(data)
                successful_scws.append(scw_id)
                print(f"Data retrieved for SCW {scw_id}")
            except Exception as e:
                print(f"Error retrieving data for SCW {scw_id}: {e}")

        if _disp.is_complete:
            n_complete += 1

    print(f"Complete {n_complete} / {len(disp_by_date)}")

    if n_complete == len(disp_by_date):
        print("All SCWs complete! Done.")
        break
    print("Not done, waiting for SCWs to finish...")
'''

Trying SCW 004500180010.001 with duration 2162.001118948683 for 2003-02
Trying SCW 010200140010.001 with duration 2199.001089250669 for 2003-08
Trying SCW 017000770010.001 with duration 1799.0010403795168 for 2004-03
Trying SCW 023900810010.001 with duration 1799.0010227775201 for 2004-09
Trying SCW 030000080010.001 with duration 1799.0009888308123 for 2005-03
Trying SCW 035200470010.001 with duration 3571.001977007836 for 2005-09
Trying SCW 036500830010.001 with duration 1800.0010682735592 for 2005-10
Trying SCW 042200600010.001 with duration 1499.0008520660922 for 2006-03
Trying SCW 048300560010.001 with duration 1514.0009284950793 for 2006-09
Trying SCW 054100190010.001 with duration 1800.0010336982086 for 2007-03
Trying SCW 060500110010.001 with duration 2000.0011452008039 for 2007-09
Trying SCW 066600320010.001 with duration 1935.0011545233428 for 2008-03
Trying SCW 072800220010.001 with duration 1261.0008203191683 for 2008-09
Trying SCW 072800860010.001 with duration 1931.0013622

JEM-X

In [7]:
disp_by_date = {} 
data_by_date = {}
successful_scws = []

disp = DispatcherAPI(url="https://www.astro.unige.ch/mmoda/dispatch-data", instrument="mock")

while True:
    image_results = []

    for year_month in sorted(set(year_months)):

        filtered_scws = [
            (scw[i], isot_start_times[i], isot_end_times[i], duration_seconds[i])
            for i, year_month_in_list in enumerate(year_months)
            if year_month_in_list == year_month
        ]

        if not filtered_scws:
            # print(f"No SCWs found for {year_month}")
            continue

        # Add filtering for not choosing the first 5 and last 10 SCWs of each revolution
        rrrr_max_pppp = defaultdict(int)
        for scwgr in filtered_scws:
            rrrr = scwgr[0][:4]  
            pppp = int(scwgr[0][4:8])
            rrrr_max_pppp[rrrr] = max(rrrr_max_pppp[rrrr], pppp)

        filtered_scws = [
            scwind for scwind in filtered_scws
            if 5 < int(scwind[0][4:8]) < rrrr_max_pppp[scwind[0][:4]] - 10
        ]

        # Sort SCWs by duration (smallest first)
        filtered_scws.sort(key=lambda x: x[3]) 

        if not filtered_scws:
            # print(f"No SCWs found for {year_month} after filtering for pointings")
            continue
        # else:
            # print(f"{len(filtered_scws)} SCWs found for {year_month} after filtering for pointings")

        # Process SCWs for the current year-month
        for scw_id, start_time, end_time, duration in filtered_scws:
            print(f"Trying SCW {scw_id} with duration {duration} for {year_month}")

            par_dict = {
                "RA": crab_ra,
                "DEC": crab_dec,
                "E1_keV": "3",
                "E2_keV": "15",
                "T_format": "isot",
                'T1': start_time,
                'T2': end_time,
                "detection_threshold": "5",
                "instrument": "jemx",
                "osa_version": "OSA11.2",
                "product": "jemx_image",
                "product_type": "Real",
                "scw_list": [scw_id],
            }

            if scw_id not in disp_by_date:
                disp_by_date[scw_id] = DispatcherAPI(url="https://www.astro.unige.ch/mmoda/dispatch-data", instrument="mock", wait=False)

            _disp = disp_by_date[scw_id]

            data = data_by_date.get(scw_id, None)

            if data is None and not _disp.is_failed:
                try:
                    if not _disp.is_submitted:
                        data = _disp.get_product(**par_dict, silent=True)
                    else:
                        _disp.poll()

                    if not _disp.is_complete:
                        break  # Continue to the next SCW if not complete

                    data = _disp.get_product(**par_dict, silent=True)
                    data_by_date[scw_id] = data
                    image_results.append(data)
                    successful_scws.append(scw_id)
                    # print(f"Query successful for SCW {scw_id}")
                    break  # Stop trying other SCWs for this month if one is successful

                except Exception as e:
                    # print(f"Query failed for SCW {scw_id}: {e}")
                    continue  # Try the next SCW

    # Now, poll all SCWs that have been submitted
    print("Polling for completion of all SCWs.")
    n_complete = 0

    # Poll all SCWs that have been submitted (skip already completed ones)
    for scw_id, _disp in disp_by_date.items():
        if _disp.is_complete and scw_id not in data_by_date:
            try:
                data = _disp.get_product(silent=True)
                data_by_date[scw_id] = data
                image_results.append(data)
                successful_scws.append(scw_id)
                print(f"Data retrieved for SCW {scw_id}")
            except Exception as e:
                print(f"Error retrieving data for SCW {scw_id}: {e}")

        if _disp.is_complete:
            n_complete += 1

    print(f"Complete {n_complete} / {len(disp_by_date)}")

    if n_complete == len(disp_by_date):
        print("All SCWs complete! Done.")
        break
    print("Not done, waiting for SCWs to finish...")

Trying SCW 004500180010.001 with duration 2162.001118948683 for 2003-02
Trying SCW 010200140010.001 with duration 2199.001089250669 for 2003-08
Trying SCW 017000770010.001 with duration 1799.0010403795168 for 2004-03
Trying SCW 023900810010.001 with duration 1799.0010227775201 for 2004-09
Trying SCW 030000080010.001 with duration 1799.0009888308123 for 2005-03
Trying SCW 035200470010.001 with duration 3571.001977007836 for 2005-09
Trying SCW 036500830010.001 with duration 1800.0010682735592 for 2005-10
Trying SCW 042200600010.001 with duration 1499.0008520660922 for 2006-03
Trying SCW 048300560010.001 with duration 1514.0009284950793 for 2006-09
Trying SCW 054100190010.001 with duration 1800.0010336982086 for 2007-03
Trying SCW 060500110010.001 with duration 2000.0011452008039 for 2007-09
Trying SCW 066600320010.001 with duration 1935.0011545233428 for 2008-03
Trying SCW 072800220010.001 with duration 1261.0008203191683 for 2008-09
Trying SCW 072800860010.001 with duration 1931.0013622

KeyboardInterrupt: 

Only save FITS corresponding to the ones already done for other energy ranges

In [18]:
import os

scw_to_match1 = []
scw_to_match2 = []
scw_to_match3 = []
directory1 = "../data/CrabIMG_FITS_15_30/"
directory2 = "../data/CrabIMG_FITS_30_60/"
directory3 = "../data/CrabIMG_FITS_3_15/"

for filename in os.listdir(directory1):
    if filename.endswith(".fits"):
        scw_to_match1.append(filename[:16])
for filename in os.listdir(directory2):
    if filename.endswith(".fits"):
        scw_to_match2.append(filename[:16])
for filename in os.listdir(directory3):
    if filename.endswith(".fits"):
        scw_to_match3.append(filename[:16])

print(scw_to_match1 == scw_to_match2 == scw_to_match3)


True


Save FITS

In [16]:
new_results = []
new_scws = []
for scw_id, data in data_by_date.items():
    if data is not None:
        new_results.append(data)
        new_scws.append(scw_id)

In [17]:
# Save FITS files
for i, data in enumerate(new_results):
    if new_scws[i] in scw_to_match1:
        im = OdaImage(data)
        im.write_fits(f"../data/CrabIMG_FITS_3_15/{new_scws[i]}")

No instrument in data collection
No instrument in data collection
No instrument in data collection
No instrument in data collection
No instrument in data collection
No instrument in data collection
No instrument in data collection
No instrument in data collection


In [25]:
data_collection = new_results[0]

data_collection.mosaic_image_0_mosaic.show()
print(data_collection.mosaic_image_0_mosaic.show_meta())
data_collection.dispatcher_catalog_1.table
header = data_collection.mosaic_image_0_mosaic.get_data_unit(2).header
header

------------------------------
name: mosaic_image
meta_data dict_keys(['product', 'instrument', 'src_name', 'query_parameters'])
number of data units 7
------------------------------
data uniti 0 ,name: PRIMARY
data uniti 1 ,name: GROUPING
data uniti 2 ,name: JMX2-MOSA-IMA
data uniti 3 ,name: JMX2-MOSA-IMA
data uniti 4 ,name: JMX2-MOSA-IMA
data uniti 5 ,name: JMX2-MOSA-IMA
data uniti 6 ,name: JMX2-MOSA-IMA
------------------------------
product : mosaic
instrument : jemx
src_name : 
query_parameters : None
------------------------------
None


{'BASETYPE': 'DAL_ARRAY',
 'BITPIX': -64,
 'BSCALE': 1,
 'BUNIT': 'counts/cm2/s',
 'BZERO': 0,
 'CD1_1': -0.026,
 'CD1_2': 0.0,
 'CD2_1': 0.0,
 'CD2_2': 0.026,
 'CHANMAX': 158,
 'CHANMIN': 46,
 'CHANTYPE': 'PI',
 'CHECKSUM': 'U6pVX4oSU4oSU4oS',
 'COMMENT': 'STAMP : This is the Intensity map',
 'CONFIGUR': 'osa_2021-08-27T14:27:00',
 'CREATOR': 'j_ima_mosaic 11.0.0',
 'CRPIX1': 184.0,
 'CRPIX2': 184.0,
 'CRVAL1': 87.7201690673828,
 'CRVAL2': 24.003963470459,
 'CTYPE1': 'RA---TAN',
 'CTYPE2': 'DEC--TAN',
 'CUNIT1': 'deg',
 'CUNIT2': 'deg',
 'DATASUM': '2059974729',
 'DATE': '2025-03-31T13:24:29',
 'DATE-END': 'UTC_format',
 'DATE-OBS': 'UTC_format',
 'EQUINOX': 2000.0,
 'EVNTYPES': 0,
 'EXTNAME': 'JMX2-MOSA-IMA',
 'EXTREL': '10.7',
 'EXTVER': 1,
 'E_MAX': 14.8800001144409,
 'E_MEAN': 8.96000003814697,
 'E_MIN': 3.03999996185303,
 'GCOUNT': 1,
 'GRPID1': 1,
 'HDUCLAS1': 'IMAGE',
 'HDUCLASS': 'OGIP',
 'HDUDOC': 'ISDC-JEMX ADD',
 'HDUVERS': '1.1.0',
 'IMADESCR': 'NOPIF',
 'IMATYPE': 'RECONS

## LIGHT CURVE

In [14]:
api_cat={
    "cat_frame": "fk5",
    "cat_coord_units": "deg",
    "cat_column_list": [
        [0],
        ["Crab"],
        [125.4826889038086],
        [crab_ra],
        [crab_dec],
        [-32768],
        [2],
        [0],
        [0.0002800000074785203]],
    "cat_column_names": [
        "meta_ID",
        "src_names",
        "significance",
        "ra",
        "dec",
        "NEW_SOURCE",
        "ISGRI_FLAG",
        "FLAG",
        "ERR_RAD"
    ],
    "cat_column_descr":
        [
            ["meta_ID", "<i8"],
            ["src_names", "<U11"],
            ["significance", "<f8"],
            ["ra", "<f8"],
            ["dec", "<f8"],
            ["NEW_SOURCE", "<i8"],
            ["ISGRI_FLAG", "<i8"],
            ["FLAG", "<i8"],
            ["ERR_RAD", "<f8"]
        ],
    "cat_lat_name": "dec",
    "cat_lon_name": "ra"
}

15 - 30 keV

In [ ]:
'''lc_disp_by_date = {} 
lc_data_by_date = {}
successful_lc_scws = []

disp = DispatcherAPI(url="https://www.astro.unige.ch/mmoda/dispatch-data", instrument="mock")

while True:
    lc_results = []

    for year_month in sorted(set(year_months)):

        filtered_scws = [
            (scw[i], isot_start_times[i], isot_end_times[i], duration_seconds[i])
            for i, year_month_in_list in enumerate(year_months)
            if year_month_in_list == year_month
        ]

        if not filtered_scws:
            print(f"No SCWs found for {year_month}")
            continue

        # Add filtering for not choosing the first 5 and last 10 SCWs of each revolution
        rrrr_max_pppp = defaultdict(int)
        for scwgr in filtered_scws:
            rrrr = scwgr[0][:4]  
            pppp = int(scwgr[0][4:8])
            rrrr_max_pppp[rrrr] = max(rrrr_max_pppp[rrrr], pppp)

        filtered_scws = [
            scwind for scwind in filtered_scws
            if 5 < int(scwind[0][4:8]) < rrrr_max_pppp[scwind[0][:4]] - 10
        ]

        # Sort SCWs by duration (smallest first)
        filtered_scws.sort(key=lambda x: x[3]) 

        if not filtered_scws:
            print(f"No SCWs found for {year_month} after filtering for pointings")
            continue
        else:
            print(f"{len(filtered_scws)} SCWs found for {year_month} after filtering for pointings")
        

        # Process SCWs for the current year-month
        for scw_id, start_time, end_time, duration in filtered_scws:
            print(f"Trying SCW {scw_id} with duration {duration} for {year_month}")

            par_dict = {
                "RA": crab_ra,
                "DEC": crab_dec,
                "E1_keV": "30", 
                "E2_keV": "60",
                "T_format": "isot",
                'T1': start_time,
                'T2': end_time,
                "time_bin": duration, 
                "instrument": "isgri",
                "osa_version": "OSA11.2",
                "product": "isgri_lc",
                "product_type": "Real",
                "scw_list": [scw_id],
                #'token': disp.disable_email_token(oda_api.token.discover_token()),
                'selected_catalog': json.dumps(api_cat)
            }

            if scw_id not in lc_disp_by_date:
                lc_disp_by_date[scw_id] = DispatcherAPI(url="https://www.astro.unige.ch/mmoda/dispatch-data", instrument="mock", wait=False)

            _disp = lc_disp_by_date[scw_id]

            lc_data = lc_data_by_date.get(scw_id, None)

            if lc_data is None and not _disp.is_failed:
                try:
                    if not _disp.is_submitted:
                        lc_data = _disp.get_product(**par_dict, silent=True)
                    else:
                        _disp.poll()

                    if not _disp.is_complete:
                        break  # Continue to the next SCW if not complete

                    lc_data = _disp.get_product(**par_dict, silent=True)
                    lc_data_by_date[scw_id] = lc_data
                    lc_results.append(lc_data)
                    successful_lc_scws.append(scw_id)
                    print(f"Query successful for SCW {scw_id}")
                    break  # Stop trying other SCWs for this month if one is successful

                except Exception as e:
                    print(f"Query failed for SCW {scw_id}: {e}")
                    continue  # Try the next SCW

        print(f"Finished processing {year_month}. Moving to the next month.")

    # Now, poll all SCWs that have been submitted
    print("Polling for completion of all SCWs.")
    n_complete = 0

    # Poll all SCWs that have been submitted (skip already completed ones)
    for scw_id, _disp in lc_disp_by_date.items():
        if _disp.is_complete and scw_id not in lc_data_by_date:
            try:
                lc_data = _disp.get_product(silent=True)
                lc_data_by_date[scw_id] = lc_data
                lc_results.append(lc_data)
                successful_lc_scws.append(scw_id)
                print(f"Data retrieved for SCW {scw_id}")
            except Exception as e:
                print(f"Error retrieving data for SCW {scw_id}: {e}")

        if _disp.is_complete:
            n_complete += 1

    print(f"Complete {n_complete} / {len(disp_by_date)}")

    if n_complete == len(lc_disp_by_date):
        print("All SCWs complete! Done.")
        break
    print("Not done, waiting for SCWs to finish...")
'''

172 SCWs found for 2003-02 after filtering for pointings
Trying SCW 004500180010.001 with duration 2162.001118948683 for 2003-02
Query successful for SCW 004500180010.001
Finished processing 2003-02. Moving to the next month.
No SCWs found for 2003-03 after filtering for pointings
No SCWs found for 2003-04 after filtering for pointings
19 SCWs found for 2003-08 after filtering for pointings
Trying SCW 010200140010.001 with duration 2199.001089250669 for 2003-08
Query successful for SCW 010200140010.001
Finished processing 2003-08. Moving to the next month.
25 SCWs found for 2004-03 after filtering for pointings
Trying SCW 017000770010.001 with duration 1799.0010403795168 for 2004-03
Query successful for SCW 017000770010.001
Finished processing 2004-03. Moving to the next month.
No SCWs found for 2004-04 after filtering for pointings
No SCWs found for 2004-08 after filtering for pointings
79 SCWs found for 2004-09 after filtering for pointings
Trying SCW 023900810010.001 with duration 1

30 - 60 keV

In [ ]:
lc_disp_by_date = {} 
lc_data_by_date = {}
successful_lc_scws = []

disp = DispatcherAPI(url="https://www.astro.unige.ch/mmoda/dispatch-data", instrument="mock")

# Keep track of the year-months that have already been processed
#processed_year_months = set()

while True:
    lc_results = []

    for year_month in sorted(set(year_months)):
        #if year_month in processed_year_months:
            #print(f"Already processed {year_month}, skipping.")
            #continue  # Skip if this year-month has already been processed

        filtered_scws = [
            (scw[i], isot_start_times[i], isot_end_times[i], duration_seconds[i])
            for i, year_month_in_list in enumerate(year_months)
            if year_month_in_list == year_month
        ]

        #if not filtered_scws:
            #print(f"No SCWs found for {year_month}")
            #continue

        # Add filtering for not choosing the first 5 and last 10 SCWs of each revolution
        rrrr_max_pppp = defaultdict(int)
        for scwgr in filtered_scws:
            rrrr = scwgr[0][:4]  
            pppp = int(scwgr[0][4:8])
            rrrr_max_pppp[rrrr] = max(rrrr_max_pppp[rrrr], pppp)

        filtered_scws = [
            scwind for scwind in filtered_scws
            if 5 < int(scwind[0][4:8]) < rrrr_max_pppp[scwind[0][:4]] - 10
        ]

        # Sort SCWs by duration (smallest first)
        filtered_scws.sort(key=lambda x: x[3]) 

        #if not filtered_scws:
            #print(f"No SCWs found for {year_month} after filtering for pointings")
            #continue
        #else:
            #print(f"{len(filtered_scws)} SCWs found for {year_month} after filtering for pointings")

        # Process SCWs for the current year-month
        for scw_id, start_time, end_time, duration in filtered_scws:
            print(f"Trying SCW {scw_id} with duration {duration} for {year_month}")

            par_dict = {
                "RA": crab_ra,
                "DEC": crab_dec,
                "E1_keV": "30",
                "E2_keV": "60",
                "T_format": "isot",
                'T1': start_time,
                'T2': end_time,
                "time_bin": duration,
                "instrument": "isgri",
                "osa_version": "OSA11.2",
                "product": "isgri_lc",
                "product_type": "Real",
                "scw_list": [scw_id],
                'selected_catalog': json.dumps(api_cat)
            }

            if scw_id not in lc_disp_by_date:
                lc_disp_by_date[scw_id] = DispatcherAPI(url="https://www.astro.unige.ch/mmoda/dispatch-data", instrument="mock", wait=False)

            _disp = lc_disp_by_date[scw_id]

            data = lc_data_by_date.get(scw_id, None)

            if data is None and not _disp.is_failed:
                try:
                    if not _disp.is_submitted:
                        data = _disp.get_product(**par_dict, silent=True)
                    else:
                        _disp.poll()

                    if not _disp.is_complete:
                        break  # Continue to the next SCW if not complete

                    data = _disp.get_product(**par_dict, silent=True)
                    lc_data_by_date[scw_id] = data
                    image_results.append(data)
                    successful_scws.append(scw_id)
                    #print(f"Query successful for SCW {scw_id}")
                    break  # Stop trying other SCWs for this month if one is successful

                except Exception as e:
                    #print(f"Query failed for SCW {scw_id}: {e}")
                    continue  # Try the next SCW

        # Mark the year-month as processed once all SCWs are attempted
        #processed_year_months.add(year_month)

        #print(f"Finished processing {year_month}. Moving to the next month.")

    # Now, poll all SCWs that have been submitted
    print("Polling for completion of all SCWs.")
    n_complete = 0

    # Poll all SCWs that have been submitted (skip already completed ones)
    for scw_id, _disp in lc_disp_by_date.items():
        if _disp.is_complete and scw_id not in lc_data_by_date:
            try:
                data = _disp.get_product(silent=True)
                lc_data_by_date[scw_id] = data
                lc_results.append(data)
                successful_lc_scws.append(scw_id)
                print(f"Data retrieved for SCW {scw_id}")
            except Exception as e:
                print(f"Error retrieving data for SCW {scw_id}: {e}")

        if _disp.is_complete:
            n_complete += 1

    print(f"Complete {n_complete} / {len(lc_disp_by_date)}")

    if n_complete == len(lc_disp_by_date):
        print("All SCWs complete! Done.")
        break
    print("Not done, waiting for SCWs to finish...")


Save FITS

In [ ]:
# Save FITS files
for i, data in enumerate(lc_results):
    lc = OdaLightCurve(data)
    lc.write_fits("Crab")
    new_filename = f"../data/CrabLC_FITS_30_60/{successful_lc_scws[i]}"
    os.rename("IBIS_lc_Crab.fits", new_filename)